In [1]:
from pathlib import Path
import multiprocessing as mp
import itertools
import operator
import pydicom
import pandas as pd
import numpy as np

num_workers = mp.cpu_count() - 1

In [2]:
p = Path("/nfs/masi/khanms/massion/test")

dcm_list = list(p.glob("**/*.dcm"))

In [3]:
dcm_list[0]

PosixPath('/nfs/masi/khanms/massion/test/10291207324/54922574/402/1168.dcm')

# Instance Check

In [14]:
def dcm_instance(dcm_file):
    '''
    For each dcm file -> (Subject, Session, Instance, InstanceNumber)
    '''
    ds = pydicom.dcmread(str(dcm_file))
    try:
        return( Path(dcm_file).parts[-4], Path(dcm_file).parts[-3], Path(dcm_file).parts[-2], int(ds[0x20, 0x13].value))
    except:
        return( Path(dcm_file).parts[-4], Path(dcm_file).parts[-3], Path(dcm_file).parts[-2], np.nan )


def instance_check(dcm_root_folder):
    # get a list of all of the DICOM files in the root folder
    dcm_list = list(Path(dcm_root_folder).glob("**/*.dcm"))
    # run `dcm_instance` on each DICOM file in the list (parallelized)
    pool = mp.Pool(processes=num_workers)
    l = pool.map(dcm_instance, dcm_list)
    pool.close()
    pool.join()
    # take output of this info and create a df
    #key, mx, mn, ct = [], [], [], []
    # groupby subject_id, session and instance
    key = [k for k,v in itertools.groupby(l, key=lambda x:(x[0], x[1], x[2]))]
    # get max, min instance num for each subj, session, instance group
    mx = [max(v)[-1] for k,v in itertools.groupby(l, key=lambda x:(x[0], x[1], x[2]))]
    mn = [min(v)[-1] for k,v in itertools.groupby(l, key=lambda x:(x[0], x[1], x[2]))]
    # get the num of dcm image files for each instance
    ct = [len(list(v)) for k,v in itertools.groupby(l, key=lambda x:(x[0], x[1], x[2]))]
    
    #generate dataframe
    df = pd.DataFrame( zip(key, ct, mx, mn) , columns=['key', 'dcmN', 'max_instN', 'min_instN'])
    df['subject'], df['session'], df['inst'] = zip(*df['key'])
    df = df.assign(instanceN = df['max_instN'] - df['min_instN'] + 1,
                   delta_dcmN_instN = df['max_instN'] - df['min_instN'] + 1 - df['dcmN'])
    df = df.groupby(['subject', 'session']).apply(lambda x: x.loc[x.dcmN.idxmax(),['inst','dcmN','instanceN', 'delta_dcmN_instN']]).reset_index()
    # indicate duplicate if instanceN is 0.5 of dcmN; `np.where` is optimized so negligible time to run
    df['dupe_inst'] = np.where(df.instanceN / df.dcmN == 0.5, 1, 0)
    
    return df

In [19]:
%%timeit

instance_check("/nfs/masi/khanms/massion/test")

4.38 s ± 83.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
test = instance_check("/nfs/masi/khanms/massion/test")
test

,subject,session,inst,dcmN,instanceN,delta_dcmN_instN,dupe_inst
0,10291207324,54741763,new_max,1408,704,-704,1
1,10291207324,54922574,new_max,554,554,0,0
2,10291207324,56876118,new_max,391,391,0,0


> TODO: Distribution of instanceN / dcmN for NLST. I hypothesize 0 and 0.5 are fine, curious what the other ratios are all about.

# Slice Distance Check

In [16]:
def dcm_slicedist(dcm_file):
    '''
    For each dcm file -> (Subject, Session, Instance, SliceLocation)
    '''
    ds = pydicom.dcmread(str(dcm_file))
    try:
        return( Path(dcm_file).parts[-4], Path(dcm_file).parts[-3], Path(dcm_file).parts[-2], float(ds.SliceLocation) )
    except:
        return( Path(dcm_file).parts[-4], Path(dcm_file).parts[-3], Path(dcm_file).parts[-2], np.nan )

# Alternative to `slice_analyzer`
def slice_loc_delta(ds_list):
    ds_sort = sorted(ds_list, reverse=True)
    res = True
    for i in range(0, len(ds_sort) - 2):
        if not abs(
            (ds_sort[i] - ds_sort[i + 1]) - (ds_sort[i + 1] - ds_sort[i + 2])
        ) < (ds_sort[0] - ds_sort[1]):
            res = False
    return res

def slice_analyzer(slice_list):
    """
    # inspired by https://www.geeksforgeeks.org/python-generate-successive-element-difference-list/
    This is a drop-in replacement for the `slice_loc_delta` function. Can use either. I'm not sure if one is faster than the other.
    """
    # sort from biggest to smallest
    l_sort = sorted(slice_list, reverse=True)
    # calculate dist b/w 0th and 1st element of sorted list
    initial_delta = l_sort[0] - l_sort[1]
    # calc diff b/w i-th and i+1-th element
    tmp = list(map(operator.sub, l_sort[1:], l_sort[:-1]))
    # calc diff b/w above and the i+1th and i+2th-element
    eval_list = [abs(i) for i in list(map(operator.sub, tmp[1:], tmp[:-1])) ]
    #boolean to make sure all deltas are less than delta b/w 0th and 1st elements
    check_bool = all(x < initial_delta for x in eval_list)
    return int(check_bool)

def tuple_extractor(tuple_list):
    l = [(i4) for i1, i2, i3, i4 in tuple_list]
    return l

def slicedist_check(dcm_root_folder):
    # get a list of all of the DICOM files in the root folder
    dcm_list = list(Path(dcm_root_folder).glob("**/*.dcm"))
    # run `dcm_instance` on each DICOM file in the list (parallelized)
    pool = mp.Pool(processes=num_workers)
    l = pool.map(dcm_slicedist, dcm_list)
    pool.close()
    pool.join()
    key = [k for k,v in itertools.groupby(sorted(l), key=lambda x:(x[0], x[1], x[2]))]
    tmp = [list(v) for k,v in itertools.groupby(sorted(l), key=lambda x:(x[0], x[1], x[2]))]
    val_list = list( map(tuple_extractor, tmp) )
    key_val = zip(key, val_list)
    slice_bool = [slice_loc_delta(i[1]) for i in key_val]    # can replace slice_analyzer with slice_loc_delta

    df = pd.DataFrame( zip( key, slice_bool ), columns=['key', 'distance_check'] )
    df['subject'], df['session'], df['inst'] = zip(*df['key'])
    return df[['subject', 'session', 'inst', 'distance_check']]

In [18]:
%%timeit

slicedist_check("/nfs/masi/khanms/massion/test")

4.3 s ± 19.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
dist_check = slicedist_check("/nfs/masi/khanms/massion/test")
dist_check

,subject,session,inst,distance_check
0,10291207324,54741763,502,False
1,10291207324,54741763,503,False
2,10291207324,54741763,504,False
3,10291207324,54741763,505,False
4,10291207324,54741763,506,False
5,10291207324,54741763,new_max,False
6,10291207324,54922574,402,True
7,10291207324,54922574,403,False
8,10291207324,54922574,404,False
9,10291207324,54922574,405,False


1. Riqiang's implementation only performs slice distance check (`sliceDis_fold`) on the `new_max` instances. Why?
    1. Similarly, instance check (`instanceN_fold`) also returns values corresponding to `new_max` instances. Why?
    2. Worth correcting or including discussion on this design choice in the paper.
2. Also, I don't see where the $\epsilon$, i.e. threshold tolerance is specified in the function.
3. The current implementation on GitHub is different from what is described in the paper:

```python
#Relevant part from current code: QA_tool.py
def dcm_slicedistance(...):
    ...
    ds_sort = sorted(ds_list, reverse=True)
    res = True
    for i in range(0, len(ds_sort) - 2):
        if not abs(
            (ds_sort[i] - ds_sort[i + 1]) - (ds_sort[i + 1] - ds_sort[i + 2])
        ) < (ds_sort[0] - ds_sort[1]):
            res = False
    return res
```

But the paper uses the following:

$$C_{3}=\sum_{i} 1\left\{s d_{i}<\varepsilon\right\}$$

I would propose that the following code is perhaps more in line with what is being described in the paper:

```python
def slice_analyzer(slice_list, epsilon=0.5): # or some sensible default
    """
    Takes a list of slice location values. Sorts these values from largest to smallest and calculates the difference b/w consecutive DICOM images (delta).
    For each instance, if the delta b/w 1 or more pairs of consecutive DICOMs exceeds our tolerance level (epsilon) - returns False/0, else returns True/1.
    """
    # sort from largest to smallest slice location
    l_sort = sorted(slice_list, reverse=True)
    # calc diff b/w slice location value b/w i-th and i+1-th element
    tmp = list(map(operator.sub, l_sort[1:], l_sort[:-1]))
    #boolean to make sure all deltas are less than delta b/w 0th and 1st elements
    check_bool = all(x < epsilon for x in tmp)
    return int(check_bool)    # returns 1 if pass, 0 if fail
```

TODO:
- Distribution of SD values in NLST (graph vs summary stats)
    - What is a sensible $\epsilon$ (tolerance)?
- Specify tolerance used in our analysis.
- Speed - we talk about efficiency in the paper. How long does it take to run?